# **Store Data to Vector Store (SIKEPO)**

Ini cara untuk storing ke Redis, tapi untuk [Load](#load) Document beda-beda untuk tiap data BI, OJK, dan SIKEPO. Jadi buat sendiri function `extract_all_documents_in_directory` nya

In [1]:
import warnings
warnings.filterwarnings("ignore")

## **Config**

In [2]:
from utils.config import get_config
from utils.models import ModelName, get_model

config = get_config()

## **Define Model**

In [4]:
from utils.models import ModelName, LLMModelName, EmbeddingModelName, get_model

model_name = ModelName.OPENAI
llm_model, embed_model = get_model(model_name=model_name, config=config, llm_model_name=LLMModelName.GPT_4O_MINI, embedding_model_name=EmbeddingModelName.EMBEDDING_3_SMALL)

## **Storing Ketentuan**

In [5]:
import pandas as pd
import time

month_translation = {
    'Januari': 'January', 'Februari': 'February', 'Maret': 'March', 'April': 'April', 'Mei': 'May', 'Juni': 'June', 
    'Juli': 'July', 'Agustus': 'August', 'September': 'September', 'Oktober': 'October', 'November': 'November', 'Desember': 'December'
}

def translate_month(date_str):
    for indonesian, english in month_translation.items():
        date_str = date_str.replace(indonesian, english)
    return date_str

In [6]:
from langchain.document_loaders import DataFrameLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import datetime

ketentuan_terkait = pd.read_excel("./data/ketentuan_terkait_all_peraturan.xlsx")
ketentuan_terkait['Tanggal Ketentuan'] = ketentuan_terkait['Tanggal Ketentuan'].apply(translate_month)
ketentuan_terkait['Tanggal Ketentuan'] = pd.to_datetime(ketentuan_terkait['Tanggal Ketentuan'], format='%d %B %Y')

# Extract the year and add it as a new column
# ketentuan_terkait['Tahun Ketentuan'] = ketentuan_terkait['Tanggal Ketentuan'].dt.year
# ketentuan_terkait['Tanggal Ketentuan Unix'] = ketentuan_terkait['Tanggal Ketentuan'].apply(lambda x: int(time.mktime(x.timetuple())))

ketentuan_terkait['Tanggal Ketentuan'] = ketentuan_terkait['Tanggal Ketentuan'].dt.strftime('%Y/%m/%d')

# Convert the Year column to int
# ketentuan_terkait['Tahun Ketentuan'] = ketentuan_terkait['Tahun Ketentuan'].astype(int)

ketentuan_terkait['Referensi'].fillna("-", inplace=True)
ketentuan_terkait.head()

,Nomor Ketentuan,Ketentuan,Referensi,Kodifikasi Ketentuan,Jenis Ketentuan,Judul Ketentuan,Tanggal Ketentuan,Page Content
0,11/1/PBI/2009,Pasal 2,-,Bank Umum Konvensional,PBI,Bank Umum,2009/01/22,Nomor Ketentuan: 11/1/PBI/2009\nKetentuan: Pas...
1,11/1/PBI/2009,Pasal 3,-,Bank Umum Konvensional,PBI,Bank Umum,2009/01/22,Nomor Ketentuan: 11/1/PBI/2009\nKetentuan: Pas...
2,11/1/PBI/2009,Pasal 4,-,Bank Umum Konvensional,PBI,Bank Umum,2009/01/22,Nomor Ketentuan: 11/1/PBI/2009\nKetentuan: Pas...
3,11/1/PBI/2009,Pasal 5,-,Bank Umum Konvensional,PBI,Bank Umum,2009/01/22,Nomor Ketentuan: 11/1/PBI/2009\nKetentuan: Pas...
4,11/1/PBI/2009,Pasal 6,-,Bank Umum Konvensional,PBI,Bank Umum,2009/01/22,Nomor Ketentuan: 11/1/PBI/2009\nKetentuan: Pas...


In [7]:
ketentuan_terkait['Tanggal Ketentuan']

0        2009/01/22
1        2009/01/22
2        2009/01/22
3        2009/01/22
4        2009/01/22
            ...    
26813    1991/03/18
26814    1991/03/18
26815    1991/03/18
26816    1991/03/18
26817    1991/03/18
Name: Tanggal Ketentuan, Length: 26818, dtype: object

In [8]:
DOCUMENT="Page Content"

df_loader = DataFrameLoader(ketentuan_terkait, page_content_column=DOCUMENT)
df_document = df_loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(df_document)
print(len(docs))

56777


## Storing Docs Elastic

In [11]:
from database.vector_store.vector_store import ElasticIndexManager

vector_store_manager = ElasticIndexManager(index_name='sikepo-ketentuan-terkait', embed_model=embed_model, config=config)
vector_store_manager.store_vector_index(docs, batch_size=100)

Start loading from idx: 22000
Loaded 22001-22100 documents
Loaded 22101-22200 documents
Loaded 22201-22300 documents
Loaded 22301-22400 documents
Loaded 22401-22500 documents
Loaded 22501-22600 documents
Loaded 22601-22700 documents
Loaded 22701-22800 documents
Loaded 22801-22900 documents
Loaded 22901-23000 documents
Loaded 23001-23100 documents
Loaded 23101-23200 documents
Loaded 23201-23300 documents
Loaded 23301-23400 documents
Loaded 23401-23500 documents
Loaded 23501-23600 documents
Loaded 23601-23700 documents
Loaded 23701-23800 documents
Loaded 23801-23900 documents
Loaded 23901-24000 documents
Loaded 24001-24100 documents
Loaded 24101-24200 documents
Loaded 24201-24300 documents
Loaded 24301-24400 documents
Loaded 24401-24500 documents
Loaded 24501-24600 documents
Loaded 24601-24700 documents
Loaded 24701-24800 documents
Loaded 24801-24900 documents
Loaded 24901-25000 documents
Loaded 25001-25100 documents
Loaded 25101-25200 documents
Loaded 25201-25300 documents
Loaded 25301-

## COba2 ES

In [16]:
vector_store_manager = ElasticIndexManager(index_name='sikepo-ketentuan-terkait', embed_model=embed_model, config=config)
vectorstore = vector_store_manager.load_vector_index()

Loaded index 'sikepo-ketentuan-terkait'.


In [18]:
vectorstore.max_marginal_relevance_search("Siapa yang boleh memiliki Bank di Indonesia?")

[Document(metadata={'Nomor Ketentuan': '11/3/PBI/2009', 'Ketentuan': 'Pasal 6 ', 'Referensi': '-', 'Kodifikasi Ketentuan': 'Bank Umum Syariah', 'Jenis Ketentuan': 'PBI', 'Judul Ketentuan': 'Bank Umum Syariah', 'Tanggal Ketentuan': '2009/01/29'}, page_content='Nomor Ketentuan: 11/3/PBI/2009\nKetentuan: Pasal 6 \nmemiliki Isi ketentuan sebagai berikut: Pasal 6\n(1) Bank hanya dapat didirikan dan/atau dimiliki oleh:\na. warga negara Indonesia dan/atau badan hukum Indonesia;\nb. warga negara Indonesia dan/atau badan hukum Indonesia dengan warga negara asing dan/atau badan hukum asing secara kemitraan; atau\nc. pemerintah daerah.\n(2) Kepemilikan oleh warga negara asing dan/atau badan hukum asing sebagaimana dimaksud pada ayat (1) huruf b paling banyak sebesar 99% (sembilan puluh sembilan persen) dari modal disetor Bank.\nserta memiliki penjelasan tambahan sebagai berikut: Pasal 6\nCukup jelas.'),
 Document(metadata={'Nomor Ketentuan': '28/76/KEP/DIR/1995', 'Ketentuan': 'Pasal 7 ', 'Referen

In [20]:
from langchain.globals import set_debug
set_debug(True)

from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_core.language_models.base import BaseLanguageModel
from langchain_core.vectorstores import VectorStore


metadata_field_info = [
    AttributeInfo(
        name="Jenis Ketentuan",
        description="Jenis peraturan atau ketentuan",
        type="string",
    ),
    AttributeInfo(
        name="Judul Ketentuan",
        description="Judul peraturan atau ketentuan",
        type="string",
    ),
    AttributeInfo(
        name="Ketentuan",
        description="Pasal atau ketentuan spesifik dalam peraturan",
        type="string",
    ),
    AttributeInfo(
        name="Kodifikasi Ketentuan",
        description="Kategori kodifikasi ketentuan",
        type="string",
    ),
    AttributeInfo(
        name="Nomor Ketentuan",
        description="Nomor dari ketentuan",
        type="string",
    ),
    AttributeInfo(
        name="Referensi",
        description="Referensi terkait ketentuan",
        type="string",
    ),
    AttributeInfo(
        name="Tanggal Ketentuan",
        description="Tanggal ketika ketentuan diterbitkan",
        type="string",
    ),
]

document_content_description = "Isi Ketentuan dari Peraturan"


# ini juga gua buat satu aja jadi kalau mau make self self_query_retriever_ketentuan_terkait, tinggal pake vector_store ketentuan terkait aja
def self_query_retriever_sikepo(llm_model: BaseLanguageModel, vector_store: VectorStore):
    retriever = SelfQueryRetriever.from_llm(
        llm=llm_model,
        vectorstore=vector_store,
        document_contents=document_content_description,
        metadata_field_info=metadata_field_info,
        verbose=True,
    )
    return retriever


self_query = self_query_retriever_sikepo(llm_model, vectorstore)


In [25]:
self_query.invoke("Peraturan jenis POJK terkait bank syariah yang berlaku paling terbaru (setelah tahun 2022)?")

[chain/start] [retriever:Retriever > chain:query_constructor] Entering Chain run with input:
{
  "query": "Peraturan jenis POJK terkait bank syariah yang berlaku paling terbaru (setelah tahun 2022)?"
}
[chain/start] [retriever:Retriever > chain:query_constructor > prompt:FewShotPromptTemplate] Entering Prompt run with input:
{
  "query": "Peraturan jenis POJK terkait bank syariah yang berlaku paling terbaru (setelah tahun 2022)?"
}
[chain/end] [retriever:Retriever > chain:query_constructor > prompt:FewShotPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [retriever:Retriever > chain:query_constructor > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Your goal is to structure the user's query to match the request schema provided below.\n\n<< Structured Request Schema >>\nWhen responding use a markdown code snippet with a JSON object formatted in the following schema:\n\n```json\n{\n    \"query\": string \\ text string to compare to docume

[Document(metadata={'Nomor Ketentuan': '5 Tahun 2024', 'Ketentuan': 'Pasal 116 ', 'Referensi': '-', 'Kodifikasi Ketentuan': 'Bank Umum Konvensional', 'Jenis Ketentuan': 'POJK', 'Judul Ketentuan': 'Penetapan Status Pengawasan Dan Penanganan Permasalahan Bank Umum', 'Tanggal Ketentuan': '2024/03/25'}, page_content='b. menerima pengalihan sebagian atau seluruh aset dan/atau kewajiban unit usaha syariah pada 1 (satu) atau lebih Bank Asal yang melaksanakan kegiatan usaha secara konvensional yang memiliki unit usaha syariah.\nserta memiliki penjelasan tambahan sebagai berikut: Cukup jelas.'),
 Document(metadata={'Nomor Ketentuan': '5 Tahun 2024', 'Ketentuan': 'Pasal 116 ', 'Referensi': '-', 'Kodifikasi Ketentuan': 'Bank Umum Konvensional', 'Jenis Ketentuan': 'POJK', 'Judul Ketentuan': 'Penetapan Status Pengawasan Dan Penanganan Permasalahan Bank Umum', 'Tanggal Ketentuan': '2024/03/25'}, page_content='b. menerima pengalihan sebagian atau seluruh aset dan/atau kewajiban unit usaha syariah pad

In [40]:
self_query.config_schema

<bound method Runnable.config_schema of SelfQueryRetriever(vectorstore=<langchain_community.vectorstores.pgvector.PGVector object at 0x000001AE7B2F1880>, query_constructor=RunnableBinding(bound=FewShotPromptTemplate(input_variables=['query'], examples=[{'i': 1, 'data_source': '```json\n{{\n    "content": "Lyrics of a song",\n    "attributes": {{\n        "artist": {{\n            "type": "string",\n            "description": "Name of the song artist"\n        }},\n        "length": {{\n            "type": "integer",\n            "description": "Length of the song in seconds"\n        }},\n        "genre": {{\n            "type": "string",\n            "description": "The song genre, one of "pop", "rock" or "rap""\n        }}\n    }}\n}}\n```', 'user_query': 'What are songs by Taylor Swift or Katy Perry about teenage romance under 3 minutes long in the dance pop genre', 'structured_request': '```json\n{{\n    "query": "teenager love",\n    "filter": "and(or(eq(\\"artist\\", \\"Taylor Sw

## **Coba2 Ketentuan**

In [5]:
from retriever.retriever_sikepo.self_query_sikepo import self_query_retriever_sikepo
from langchain.globals import set_debug

set_debug(True)

self_query = self_query_retriever_sikepo(llm_model, vector_store, with_limit=True)

In [6]:
self_query.invoke("Peraturan dengan nomor ketentuan 11/1/PBI/2009 dan Jenis Ketentuan PBI?")

[chain/start] [retriever:Retriever > chain:query_constructor] Entering Chain run with input:
{
  "query": "Peraturan dengan nomor ketentuan 11/1/PBI/2009 dan Jenis Ketentuan PBI?"
}
[chain/start] [retriever:Retriever > chain:query_constructor > prompt:FewShotPromptTemplate] Entering Prompt run with input:
{
  "query": "Peraturan dengan nomor ketentuan 11/1/PBI/2009 dan Jenis Ketentuan PBI?"
}
[chain/end] [retriever:Retriever > chain:query_constructor > prompt:FewShotPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [retriever:Retriever > chain:query_constructor > llm:AzureChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Your goal is to structure the user's query to match the request schema provided below.\n\n<< Structured Request Schema >>\nWhen responding use a markdown code snippet with a JSON object formatted in the following schema:\n\n```json\n{\n    \"query\": string \\ text string to compare to document contents\n    \"filter\": string

[]

In [7]:
self_query.config_schema

<bound method Runnable.config_schema of SelfQueryRetriever(vectorstore=<langchain_community.vectorstores.pgvector.PGVector object at 0x000001AE7B2F1880>, query_constructor=RunnableBinding(bound=FewShotPromptTemplate(input_variables=['query'], examples=[{'i': 1, 'data_source': '```json\n{{\n    "content": "Lyrics of a song",\n    "attributes": {{\n        "artist": {{\n            "type": "string",\n            "description": "Name of the song artist"\n        }},\n        "length": {{\n            "type": "integer",\n            "description": "Length of the song in seconds"\n        }},\n        "genre": {{\n            "type": "string",\n            "description": "The song genre, one of "pop", "rock" or "rap""\n        }}\n    }}\n}}\n```', 'user_query': 'What are songs by Taylor Swift or Katy Perry about teenage romance under 3 minutes long in the dance pop genre', 'structured_request': '```json\n{{\n    "query": "teenager love",\n    "filter": "and(or(eq(\\"artist\\", \\"Taylor Sw

## **Storing Rekam**

In [12]:
rekam_jejak = pd.read_excel("./data/rekam_jejak_all_peraturan.xlsx")

rekam_jejak['Tanggal Ketentuan'] = rekam_jejak['Tanggal Ketentuan'].apply(translate_month)
rekam_jejak['Tanggal Ketentuan'] = pd.to_datetime(rekam_jejak['Tanggal Ketentuan'], format='%d %B %Y')
rekam_jejak['Tanggal Ketentuan'] = rekam_jejak['Tanggal Ketentuan'].dt.strftime('%Y/%m/%d')

rekam_jejak.head()

,Nomor Ketentuan,Jenis Bank,Kodifikasi Ketentuan,Jenis Ketentuan,Judul Ketentuan,Tanggal Ketentuan,Page Content
0,11/1/PBI/2009,Bank Umum Konvensional,Bank Umum Konvensional,PBI,Bank Umum,2009/01/22,Peraturan 11/1/PBI/2009\nMencabut beberapa Per...
1,13/27/PBI/2011,Bank Umum Konvensional,Bank Umum Konvensional,PBI,Perubahan Atas Peraturan Bank Indonesia Nomor ...,2011/12/28,Peraturan 13/27/PBI/2011\nMengubah beberapa Pe...
2,11/1/PBI/2009,Bank Umum Konvensional,Bank Umum Konvensional,PBI,Bank Umum,2009/01/22,Peraturan 11/1/PBI/2009\nMencabut beberapa Per...
3,11/1/PBI/2009,Bank Umum Konvensional,Bank Umum Konvensional,PBI,Bank Umum,2009/01/22,Peraturan 11/1/PBI/2009\nMencabut beberapa Per...
4,13/27/PBI/2011,Bank Umum Konvensional,Bank Umum Konvensional,PBI,Perubahan Atas Peraturan Bank Indonesia Nomor ...,2011/12/28,Peraturan 13/27/PBI/2011\nMengubah beberapa Pe...


In [13]:
DOCUMENT="Page Content"

df_loader = DataFrameLoader(rekam_jejak, page_content_column=DOCUMENT)
df_document = df_loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(df_document)
print(len(docs))

8511


In [15]:
from database.vector_store.vector_store import ElasticIndexManager

vector_store_manager = ElasticIndexManager(index_name='sikepo-rekam-jejak', embed_model=embed_model, config=config)
vector_store_manager.store_vector_index(docs, batch_size=100)

Loaded 1-100 documents
Loaded 101-200 documents
Loaded 201-300 documents
Loaded 301-400 documents
Loaded 401-500 documents
Loaded 501-600 documents
Loaded 601-700 documents
Loaded 701-800 documents
Loaded 801-900 documents
Loaded 901-1000 documents
Loaded 1001-1100 documents
Loaded 1101-1200 documents
Loaded 1201-1300 documents
Loaded 1301-1400 documents
Loaded 1401-1500 documents
Loaded 1501-1600 documents
Loaded 1601-1700 documents
Loaded 1701-1800 documents
Loaded 1801-1900 documents
Loaded 1901-2000 documents
Loaded 2001-2100 documents
Loaded 2101-2200 documents
Loaded 2201-2300 documents
Loaded 2301-2400 documents
Loaded 2401-2500 documents
Loaded 2501-2600 documents
Loaded 2601-2700 documents
Loaded 2701-2800 documents
Loaded 2801-2900 documents
Loaded 2901-3000 documents
Loaded 3001-3100 documents
Loaded 3101-3200 documents
Loaded 3201-3300 documents
Loaded 3301-3400 documents
Loaded 3401-3500 documents
Loaded 3501-3600 documents
Loaded 3601-3700 documents
Loaded 3701-3800 docum

In [16]:
from database.vector_store.vector_store import RedisIndexManager, PostgresIndexManager

# redis = RedisIndexManager(index_name='ojk', embed_model=embed_model, config=config, db_id=0)
postgre = PostgresIndexManager(index_name='sikepo-rekam-jejak', embed_model=embed_model, config=config)

# redis.delete_index()
postgre.store_vector_index(docs=docs, batch_size=500) # Kalau error 'Redis failed to connect: Index does not exist.' ubah isi start_store_idx_indexname.txt menjadi 0

Database 'vector_store' already exists.
Vector extension created successfully (if it didn't exist).


c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\langchain_community\vectorstores\pgvector.py:328: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata.Please note that filtering operators have been changed when using JSOB metadata to be prefixed with a $ sign to avoid name collisions with columns. If you're using an existing database, you will need to create adb migration for your metadata column to be JSONB and update your queries to use the new operators. 
  warn_deprecated(


Loaded 1-500 documents
Loaded 501-1000 documents
Loaded 1001-1500 documents
Loaded 1501-2000 documents
Loaded 2001-2500 documents
Loaded 2501-3000 documents
Loaded 3001-3500 documents
Loaded 3501-4000 documents
Loaded 4001-4500 documents
Loaded 4501-5000 documents
Loaded 5001-5500 documents
Loaded 5501-6000 documents
Loaded 6001-6500 documents
Loaded 6501-7000 documents
Loaded 7001-7500 documents
Loaded 7501-8000 documents
Loaded 8001-8500 documents
Loaded 8501-8511 documents


In [18]:
from retriever.retriever_sikepo.self_query_sikepo import self_query_retriever_sikepo
from langchain.globals import set_debug

set_debug(True)

vector_store = postgre.load_vector_index()
self_query = self_query_retriever_sikepo(llm_model, vector_store)

c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\langchain_community\vectorstores\pgvector.py:328: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata.Please note that filtering operators have been changed when using JSOB metadata to be prefixed with a $ sign to avoid name collisions with columns. If you're using an existing database, you will need to create adb migration for your metadata column to be JSONB and update your queries to use the new operators. 
  warn_deprecated(
